In [8]:

# -*- coding:utf-8 -*-
# Imports
import sys
import os
from cv2 import *
from Tkinter import * 
from Tkinter import Tk, Menu, Canvas
from PIL import Image, ImageTk
import tkFileDialog as filedialog
import test07_gui00


#imgname = "dataset/10_left.jpeg"
width = 430
height = 320
# Function definitions
def deleteImage(canvas):
    canvas.delete("all")
    return


def quitProgram():
    gui.destroy()

    
# Main window
gui = Tk()
# setting the size of the window
canvas = Canvas(gui, width = 400, height = 300)  
canvas.pack()  


# BROWSE BUTTON CODE
def browsefunc():
    global imgname
    imgname = filedialog.askopenfilename()
    #pathlabel.config(text=imgname)
    pathLabel = Label(gui, text=imgname, anchor=S )
    pathLabel.pack()
#pathlabel = Label()
#pathlabel.pack()


# Inside the main gui window
# Creating an object containing an image
# A canvas with borders that adapt to the image within it
def loadImage():
    #global width
    #global height
    
    
    width = 400
    height = 500    
    img = Image.open(imgname)
    im2 = img.resize((width, height), Image.NEAREST)  
    #width, height = img.size
    filename = ImageTk.PhotoImage(im2)
    canvas = Canvas(gui,height=600,width=900)
    canvas.image = filename  # <--- keep reference of your image
    canvas.create_image(0,0,anchor='nw',image=filename)
    canvas.pack()

#img = Image.open(imgname)
#canvas = Canvas(gui,height= 200,width= 300)
#canvas.pack()

#RUNNING PROGRAM
def run():
    test07_gui00.reading(imgname)
    pathlabel.config(text="sdf")
    print "******************",test07_gui00.numOfCountours
    
pathlabel = Label()
pathlabel.pack()
   
    
# Menu bar
menubar = Menu(gui)
# Adding a cascade to the menu bar:
filemenu = Menu(menubar, tearoff=0)
menubar.add_cascade(label="Files", menu=filemenu)
# Adding an option to browse for an image
filemenu.add_command(label="browse", command=browsefunc)
# Adding a load image button to the cascade menu "File"
filemenu.add_command(label="Load an image", command=loadImage)
# Adding a running program to the cascade menu "File"
filemenu.add_command(label="run code", command=run)
filemenu.add_separator()
#Adding an option to quit the window
filemenu.add_command(label="Quit", command=quitProgram)
menubar.add_separator()
menubar.add_cascade(label="?")

# Display the menu bar
gui.config(menu=menubar)
gui.title("EXTRACT TEXT FROM IMAGE")
gui.mainloop()